In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import streamlit as st
from textblob import TextBlob
from pathlib import Path
import config
import requests
import pandas as pd
import yaml

In [2]:
url = 'https://rapidapi.p.rapidapi.com/api/search/NewsSearchAPI'

def load_data(query):
    querystring = {
        'pageSize': '50',
        'q': query,
        'autoCorrect': 'true',
        'pageNumber': '1',
        'safeSearch': 'true',
        'toPublishedDate': 'null',
        'fromPublishedDate': 'null',
        }
    headers = \
        {'x-rapidapi-host': config.api_host,
         'x-rapidapi-key': config.api_key}
    response = requests.request('GET', url, headers=headers,
                                params=querystring)
    text = yaml.safe_load(response.text)
    return pd.DataFrame(text['value'])

def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment
    except:
        return None

In [3]:
df = load_data("harry potter")

In [4]:
def round_polarity(x):
    if x <= -0.75:
        return -1
    elif x <= -0.25:
        return -0.5
    elif x <= 0.25:
        return 0
    elif x <= 0.75:
        return 0.5
    return 1

def round_subjectivity(x):
    if x <= 0.125:
        return 0
    elif x <= 0.375:
        return 0.25
    elif x <= 0.625:
        return 0.5
    elif x <= 0.875:
        return 0.75
    return 1

In [5]:
df['polarity'] = df.apply(lambda row: sentiment_calc(row['body'])[0], axis=1)
df['polarity'] = df.apply(lambda row: round_polarity(row['polarity']), axis=1)
df['subjectivity'] = df.apply(lambda row: sentiment_calc(row['body'])[1], axis=1)
df['subjectivity'] = df.apply(lambda row: round_subjectivity(row['subjectivity']), axis=1)
df.head()

,id,title,url,description,body,keywords,language,isSafe,datePublished,provider,image,polarity,subjectivity
0,4715961827454341362,'The Queen's Gambit' Features a 'Harry Potter'...,https://www.cheatsheet.com/entertainment/the-q...,"Netflix's new drama, 'The Queen's Gambit,' fea...",stars Anya Taylor-Joy in the lead role of ches...,,en,True,2020-10-30T18:28:01,{'name': 'cheatsheet'},"{'url': '', 'height': 0, 'width': 0, 'thumbnai...",0.0,0.50
1,3058814450427935429,Is Harry Potter on Netflix? How to stream the ...,https://metro.co.uk/2020/10/29/is-harry-potter...,The Harry Potter movies remain as popular as e...,The JK Rowling series is also remaining popula...,,en,True,2020-10-29T16:00:20,{'name': 'metro'},"{'url': '', 'height': 0, 'width': 0, 'thumbnai...",0.0,0.50
2,3510201799281910430,"Harry Potter Isnt Coming to Netflix, Its Comin...",https://observer.com/2020/10/harry-potter-film...,The shuffling of the 'Harry Potter' films acro...,Warner Bros. Harry Potter and the Goblet of Fi...,,en,True,2020-10-29T13:44:29,{'name': 'observer'},"{'url': '', 'height': 0, 'width': 0, 'thumbnai...",0.0,0.50
3,6301044588984250422,Pirates of the Caribbean: At World's End: When...,https://www.koimoi.com/hollywood-2/pirates-of-...,Pirates Of The Caribbean is one of the topmost...,tweet\nPirates of the Caribbean: At Worlds End...,,en,True,2020-10-29T11:17:16,{'name': 'koimoi'},"{'url': '', 'height': 0, 'width': 0, 'thumbnai...",0.0,0.25
4,7072050357449097583,Canada's Trudeau emerges from months of self-i...,https://www.thespoof.com/spoof-news/world/1509...,After months of self-isolation watching Happy ...,Share\nAfter months of self-isolation watching...,,en,True,2020-10-28T16:00:49,{'name': 'thespoof'},"{'url': '', 'height': 0, 'width': 0, 'thumbnai...",0.0,0.50
